In [0]:
%pip install -qqqq -U -r requirements.txt

In [0]:
dbutils.library.restartPython()

In [0]:
import os
import sys
import json
from cookbook.tools.local_function import LocalFunctionTool
sys.path.append(os.path.abspath('/Workspace/Users/mohammedarif@prudential.com.my/multi_agent_story'))

In [0]:
# Import necessary libraries
from cookbook.config.shared.agent_storage_location import AgentStorageConfig
from cookbook.databricks_utils import get_mlflow_experiment_url
from cookbook.config import load_serializable_config_from_yaml_file
import mlflow 
import os
import inspect

# Load the Agent's storage locations
agent_storage_config: AgentStorageConfig = load_serializable_config_from_yaml_file("./configs/agent_storage_config.yaml")

# Show the Agent's storage locations
agent_storage_config.pretty_print()

# Set up the MLflow experiment
experiment_info = mlflow.set_experiment(agent_storage_config.mlflow_experiment_name)
# If running in a local IDE, set the MLflow experiment name as an environment variable
os.environ["MLFLOW_EXPERIMENT_NAME"] = agent_storage_config.mlflow_experiment_name

print(f"View the MLflow Experiment `{agent_storage_config.mlflow_experiment_name}` at {get_mlflow_experiment_url(experiment_info.experiment_id)}")

In [0]:
# Enable auto-reloading for easier development
# %load_ext autoreload
# %autoreload 2

In [0]:
# Set up UC function client
from unitycatalog.ai.core.databricks import DatabricksFunctionClient
from cookbook.tools.uc_tool import UCTool
from cookbook.tools.local_function import LocalFunctionTool

client = DatabricksFunctionClient()

# Set Unity Catalog information - make sure to use the correct catalog and schema for your environment
UC_CATALOG = "main"  # Change to your catalog
UC_SCHEMA = "default"  # Change to your schema

# Import the visualization tools from the tools module
from tools.visualization_tools import (
    create_bar_chart,
    create_line_chart,
    create_pie_chart,
    format_table_data,
    python_viz_executor,
    extract_table_from_markdown,
    generate_visualization_code
)

# Display function documentation
print("Bar Chart Function Documentation:")
print(inspect.getdoc(create_bar_chart))
print("\n" + "="*50 + "\n")

print("Line Chart Function Documentation:")
print(inspect.getdoc(create_line_chart))
print("\n" + "="*50 + "\n")

print("Pie Chart Function Documentation:")
print(inspect.getdoc(create_pie_chart))
print("\n" + "="*50 + "\n")

print("Table Formatting Function Documentation:")
print(inspect.getdoc(format_table_data))
print("\n" + "="*50 + "\n")

print("Python Visualization Executor Function Documentation:")
print(inspect.getdoc(python_viz_executor))
print("\n" + "="*50 + "\n")

# Try/except block to handle errors in function deployment
try:
    # Deploy the functions to Unity Catalog
    # Since we removed default parameters from the functions, they should deploy correctly now
    
    # Create and deploy bar chart function
    bar_chart_uc_info = client.create_python_function(
        func=create_bar_chart, 
        catalog=UC_CATALOG, 
        schema=UC_SCHEMA, 
        replace=True
    )
    print(f"Deployed bar chart function: {bar_chart_uc_info.full_name}")
    
    # Create and deploy line chart function
    line_chart_uc_info = client.create_python_function(
        func=create_line_chart, 
        catalog=UC_CATALOG, 
        schema=UC_SCHEMA, 
        replace=True
    )
    print(f"Deployed line chart function: {line_chart_uc_info.full_name}")
    
    # Create and deploy pie chart function
    pie_chart_uc_info = client.create_python_function(
        func=create_pie_chart, 
        catalog=UC_CATALOG, 
        schema=UC_SCHEMA, 
        replace=True
    )
    print(f"Deployed pie chart function: {pie_chart_uc_info.full_name}")
    
    # Create and deploy table formatting function
    table_format_uc_info = client.create_python_function(
        func=format_table_data, 
        catalog=UC_CATALOG, 
        schema=UC_SCHEMA, 
        replace=True
    )
    print(f"Deployed table format function: {table_format_uc_info.full_name}")
    
    # Create and deploy Python visualization executor
    python_viz_uc_info = client.create_python_function(
        func=python_viz_executor, 
        catalog=UC_CATALOG, 
        schema=UC_SCHEMA, 
        replace=True
    )
    print(f"Deployed Python visualization executor function: {python_viz_uc_info.full_name}")
    
    # Create and deploy markdown table extraction function
    extract_table_uc_info = client.create_python_function(
        func=extract_table_from_markdown, 
        catalog=UC_CATALOG, 
        schema=UC_SCHEMA, 
        replace=True
    )
    print(f"Deployed markdown table extraction function: {extract_table_uc_info.full_name}")
    
    # Create and deploy visualization code generator function
    viz_code_gen_uc_info = client.create_python_function(
        func=generate_visualization_code, 
        catalog=UC_CATALOG, 
        schema=UC_SCHEMA, 
        replace=True
    )
    print(f"Deployed visualization code generator function: {viz_code_gen_uc_info.full_name}")

    # Create UCTool wrappers for the deployed functions
    bar_chart_tool = UCTool(uc_function_name=bar_chart_uc_info.full_name)
    line_chart_tool = UCTool(uc_function_name=line_chart_uc_info.full_name)
    pie_chart_tool = UCTool(uc_function_name=pie_chart_uc_info.full_name)
    table_format_tool = UCTool(uc_function_name=table_format_uc_info.full_name)
    python_viz_tool = UCTool(uc_function_name=python_viz_uc_info.full_name)
    extract_table_tool = UCTool(uc_function_name=extract_table_uc_info.full_name)
    viz_code_gen_tool = UCTool(uc_function_name=viz_code_gen_uc_info.full_name)

    # Test the bar chart tool with example data
    test_data = [
        {"x": "Jan", "y": 10},
        {"x": "Feb", "y": 15},
        {"x": "Mar", "y": 7},
        {"x": "Apr", "y": 12}
    ]

    result = bar_chart_tool(
        data=json.dumps(test_data),
        title="Monthly Sales",
        x_label="Month",
        y_label="Sales ($K)",
        orientation="vertical",
        color="steelblue",
        figsize="10,6"
    )

    print("Bar Chart Test Result:")
    if "error" in result:
        print(f"Error: {result['error']}")
    else:
        print(f"Success! Image data length: {len(result['value'])}")

    # Test the Python visualization executor
    test_viz_code = """
    import matplotlib.pyplot as plt
    import numpy as np

    # Create some data
    x = np.linspace(0, 10, 100)
    y = np.sin(x)

    # Create a simple plot
    plt.figure(figsize=(8, 5))
    plt.plot(x, y, 'b-', linewidth=2)
    plt.title('Sine Wave')
    plt.xlabel('X axis')
    plt.ylabel('Y axis')
    plt.grid(True)
    """

    python_viz_result = python_viz_tool(code=test_viz_code)

    print("\nPython Visualization Test Result:")
    if "error" in python_viz_result:
        print(f"Error: {python_viz_result['error']}")
    else:
        print(f"Success! Image data available in result")

    # Create local function tools as well for use in the Visualization Agent
    print("\nCreating local function tools...")
    bar_chart_local_tool = LocalFunctionTool(func=create_bar_chart)
    line_chart_local_tool = LocalFunctionTool(func=create_line_chart)
    pie_chart_local_tool = LocalFunctionTool(func=create_pie_chart)
    table_format_local_tool = LocalFunctionTool(func=format_table_data)
    python_viz_local_tool = LocalFunctionTool(func=python_viz_executor)
    extract_table_local_tool = LocalFunctionTool(func=extract_table_from_markdown)
    viz_code_gen_local_tool = LocalFunctionTool(func=generate_visualization_code)

    print("\nCreated visualization tools as both UC Tools and Local Function Tools")
    print("These can now be imported by the Visualization Agent to create dynamic visualizations")

except Exception as e:
    print(f"Error deploying tools: {str(e)}")
    
    # Even if UC tools fail, create local tools for use in the Visualization Agent
    print("\nCreating local function tools...")
    bar_chart_local_tool = LocalFunctionTool(func=create_bar_chart)
    line_chart_local_tool = LocalFunctionTool(func=create_line_chart)
    pie_chart_local_tool = LocalFunctionTool(func=create_pie_chart)
    table_format_local_tool = LocalFunctionTool(func=format_table_data)
    python_viz_local_tool = LocalFunctionTool(func=python_viz_executor)
    extract_table_local_tool = LocalFunctionTool(func=extract_table_from_markdown)
    viz_code_gen_local_tool = LocalFunctionTool(func=generate_visualization_code)
    
    print("\nCreated local function tools that will work even without UC deployment")
    print("The Visualization Agent can use these even if UC deployment failed")